In [2]:
from tensorflow.python.keras.layers import Input,Dense,Reshape,Lambda,RNN,Concatenate,StackedRNNCells
from tensorflow.python.keras.models import Model,load_model
from tensorflow.python.keras.utils import multi_gpu_model
from tensorflow.python.keras.optimizers import Adam,SGD
from tensorflow.python.keras.callbacks import LearningRateScheduler,Callback,ModelCheckpoint,TensorBoard

import tensorflow as tf
import datetime
import os
import numpy as np
import pickle

%env PYTHONHASHSEED=0
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/

env: PYTHONHASHSEED=0


/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site

In [3]:
from sklearn.model_selection import train_test_split
#features
n2v_start = np.load("start_emb_128.npy")
n2v_end = np.load("end_emb_128.npy")
n2v_start_32 = np.load("start_emb.npy")
n2v_end_32 = np.load("end_emb.npy")
lanes = np.load("lanes.npy")
oneway = np.load("oneway.npy")
roadtype = np.load("highway.npy")
maxspeed = np.load("maxspeed.npy")
nodetype = np.load("nodetype.npy")
lanes_df = np.load("lanes_df.npy")
#target
meanspeed = np.load("meanspeed_only.npy")
z_score = np.load('z-score.npy')

#split
n2v_start_train, n2v_start_test, n2v_end_train, n2v_end_test,n2v_start_32_train,n2v_start_32_test,n2v_end_32_train,n2v_end_32_test, lanes_train, lanes_test, oneway_train, oneway_test, roadtype_train, roadtype_test, maxspeed_train, maxspeed_test, nodetype_train, nodetype_test, lanes_df_train, lanes_df_test, meanspeed_train, meanspeed_test, z_score_train, z_score_test = train_test_split(n2v_start, n2v_end,n2v_start_32,n2v_end_32, lanes,oneway,roadtype ,maxspeed,nodetype,lanes_df,meanspeed,z_score)

del n2v_start
del n2v_end
del lanes
del oneway
del roadtype
del maxspeed
del nodetype
del lanes_df
del meanspeed
del z_score

In [12]:
embinplist = list()
inplist = list()


n2v_start_inp = Input((128,))
n2v_start_reg = Dense(128,activation = "relu", use_bias = True)(n2v_start_inp)
inplist.append(n2v_start_inp)
embinplist.append(n2v_start_reg)

n2v_end_inp = Input((128,))
n2v_end_reg = Dense(128,activation = "relu", use_bias = True)(n2v_end_inp)
inplist.append(n2v_end_inp)
embinplist.append(n2v_end_reg)

lanes_inp = Input((1,))
lanes_reg= Dense(1,activation = "relu", use_bias = True)(lanes_inp)
inplist.append(lanes_inp)
embinplist.append(lanes_reg)

oneway_inp = Input((2,))
oneway_reg= Dense(2,activation = "relu", use_bias = True)(oneway_inp)
inplist.append(oneway_inp)
embinplist.append(oneway_reg)

roadtype_inp = Input((13,))
roadtype_reg= Dense(8,activation = "relu", use_bias = True)(roadtype_inp)
inplist.append(roadtype_inp)
embinplist.append(roadtype_reg)

maxspeed_inp = Input((1,))
maxspeed_reg= Dense(1,activation = "relu", use_bias = True)(maxspeed_inp)
inplist.append(maxspeed_inp)
embinplist.append(maxspeed_reg)

nodetype_inp = Input((11,))
nodetype_reg= Dense(8,activation = "relu", use_bias = True)(nodetype_inp)
inplist.append(nodetype_inp)
embinplist.append(nodetype_reg)

lanes_df_inp = Input((1,))
lanes_df_reg= Dense(1,activation = "relu", use_bias = True)(lanes_df_inp)
inplist.append(lanes_df_inp)
embinplist.append(lanes_df_reg)

combine = Concatenate()(embinplist)
proj = Dense(32,activation = "relu",use_bias = True)(combine)#proj = Dense(32,activation = "relu",use_bias = True)(combine)
proj = Dense(1,activation = "linear")(proj)

emb = Model(inplist,proj)

In [26]:
emb.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 2)]          0                                            
____________________________________________________________________________________________

In [16]:
adam=Adam(lr=0.01,beta_1=0.9,beta_2=0.999,amsgrad=True)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
emb.compile(optimizer=adam, loss='mae',metrics=['mae'])

In [18]:
history =emb.fit([n2v_start_train,n2v_end_train,lanes_train,oneway_train,roadtype_train,maxspeed_train,nodetype_train,lanes_df_train],
        meanspeed_train,
        epochs=100,batch_size=256,
        validation_data=([n2v_start_test,n2v_end_test,lanes_test,oneway_test,roadtype_test,maxspeed_test,nodetype_test,lanes_df_test],
        meanspeed_test)
       )

Train on 86992 samples, validate on 28998 samples
Epoch 1/100
86992/86992 [==============================] - 4s 42us/sample - loss: 5.6865 - mean_absolute_error: 5.6865 - val_loss: 5.5441 - val_mean_absolute_error: 5.5441
Epoch 2/100
86992/86992 [==============================] - 3s 37us/sample - loss: 4.9934 - mean_absolute_error: 4.9934 - val_loss: 5.1709 - val_mean_absolute_error: 5.1709
Epoch 3/100
86992/86992 [==============================] - 3s 37us/sample - loss: 4.6175 - mean_absolute_error: 4.6175 - val_loss: 5.1998 - val_mean_absolute_error: 5.1998
Epoch 4/100
86992/86992 [==============================] - 3s 37us/sample - loss: 4.4139 - mean_absolute_error: 4.4139 - val_loss: 5.0167 - val_mean_absolute_error: 5.0167
Epoch 5/100
86992/86992 [==============================] - 3s 37us/sample - loss: 4.2389 - mean_absolute_error: 4.2389 - val_loss: 4.7672 - val_mean_absolute_error: 4.7672
Epoch 6/100
86992/86992 [==============================] - 3s 37us/sample - loss: 4.0992 -

86992/86992 [==============================] - 3s 37us/sample - loss: 2.1785 - mean_absolute_error: 2.1785 - val_loss: 3.6977 - val_mean_absolute_error: 3.6977
Epoch 96/100
86992/86992 [==============================] - 3s 37us/sample - loss: 2.1627 - mean_absolute_error: 2.1627 - val_loss: 3.6686 - val_mean_absolute_error: 3.6686
Epoch 97/100
86992/86992 [==============================] - 3s 37us/sample - loss: 2.1594 - mean_absolute_error: 2.1594 - val_loss: 3.6859 - val_mean_absolute_error: 3.6859
Epoch 98/100
86992/86992 [==============================] - 3s 38us/sample - loss: 2.1682 - mean_absolute_error: 2.1682 - val_loss: 3.6286 - val_mean_absolute_error: 3.6286
Epoch 99/100
86992/86992 [==============================] - 3s 38us/sample - loss: 2.1475 - mean_absolute_error: 2.1475 - val_loss: 3.6681 - val_mean_absolute_error: 3.6681
Epoch 100/100
86992/86992 [==============================] - 3s 38us/sample - loss: 2.1589 - mean_absolute_error: 2.1589 - val_loss: 3.7238 - val_me

In [49]:
emb.save('/home/fyp2/Ideaer/Madrid/model/all')

In [19]:
loss_hist = history.history['loss']
mae_history = history.history["val_mean_absolute_error"]
print("min loss :"+str(np.min(loss_hist)))
print("min val mae :"+str(np.min(mae_history)))

min loss :2.1475440418323393
min val mae :3.628573


In [11]:
n2v_start_32_train

array([[-1.5621526 , -2.140005  , -1.370696  , ..., -4.470399  ,
        -0.9705669 ,  3.494547  ],
       [ 2.563424  , -5.929396  , -3.8642359 , ..., -2.864576  ,
        -1.7979022 ,  2.4272647 ],
       [-1.5728813 , -3.7323647 , -4.434676  , ..., -3.9871058 ,
        -1.6048346 , -4.865197  ],
       ...,
       [ 1.1419044 ,  6.2074    ,  0.8944777 , ..., -0.3862339 ,
        -2.0870483 , -2.8593926 ],
       [ 1.7119933 , -2.7071478 , -2.4384882 , ...,  1.8699619 ,
        -3.8077543 , -3.0337284 ],
       [-2.8112872 , -1.38895   ,  0.52061427, ...,  5.172755  ,
        -6.2895813 ,  1.0420344 ]], dtype=float32)